In [1]:
!apt-get -y update
!apt-get -y install unzip wget
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mv ./ngrok /usr/bin/ngrok
!rm -rf ngrok-stable-linux-amd64.zip
!ngrok authtoken 27uk1MTeqoUMsxLa3xISsHk727m_73D2ybvQ9LxNvURSCNiuv

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease   
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [874 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [2280 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1205 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [27.5 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1850 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1161 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [30.2 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [1285 kB]
Get:13 http://ar

In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached python_speech_features-0.6-py3-none-any.whl
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/f9/f4/ede7c643939c132b0692a737800747ce5ba0e8068af27730dfda936c9bf1/pandas-1.1.5-cp38-cp38-manylinux1_x86_64.whl (9.3 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/4d/f9/9bc5c420651e11d5ded3eef4404f907f318c242c24a36f23e22e063998bb/tensorflow-2.5.1-cp38-cp38-manylinux2010_x86_64.whl (454.5 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/d7/d9/b77dc634a7a0c0c97716ba97dd0a28cbfa6267c96f359c4f27ed71cbd284/g2p_en-2.1.0-py3-none-any.whl (3.1 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/b3/27/15edd6e1519f4e489ff50def9367a62d138baa9c000a053c67ac38156519/inflect-4.1.0-py3-none-any.whl (31 kB)
  Using cached librosa-0.7.2-py3-none-any.whl
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/77/5c/6a3aa574ed33911a0b4deb66b3b9d2e72ed4bcbfd70f52a5546395156dd3/matplotlib-3.2

In [3]:
!pip install transformers soundfile librosa moviepy pytube fastapi uvicorn starlette g2p-en tensorflow unidecode python-multipart

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/52/82/b62f139e77c70fab2763d99d83a77a27a179502430851144a1765e37f5ad/transformers-4.19.2-py3-none-any.whl (4.2 MB)
  Using cached moviepy-1.0.3-py3-none-any.whl
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/21/30/b4b72a27c2b4bca2a03a82435a5b29e6bc33a7ea7c9f277ba6ecb1dc663e/pytube-12.1.0-py3-none-any.whl (56 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/61/0f/427f0af121b226e62237e430f5bf4485e0ae1565b3f5b782613b59f30abc/fastapi-0.78.0-py3-none-any.whl (54 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/36/ab/c13847c53d0624ee5a2e19c9c8d19a8cea5f865b95d08b839fac375a9e83/uvicorn-0.17.6-py3-none-any.whl (53 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/e3/ac/6e74969e2d185bd1eab52f1e8b08b846c1d18dfa64605ba1e8f7935e4a9c/starlette-0.20.0-py3-none-any.whl (61 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/d7/

In [8]:
import os
import torch
from glob import glob

from pytube import YouTube
import moviepy.editor as mp

import torch
import librosa
import soundfile as sf
import shutil
from tqdm import tqdm


from transformers import Wav2Vec2Processor, HubertForCTC,\
                        BartForConditionalGeneration, BartTokenizer,\
                        AutoModelForSeq2SeqLM, AutoTokenizer

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("STARTING MODELS LOAD")

transcript_processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")
transcript_model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft").to(device)

print("TRANSCRIPT MODELS LOADED")

paraphrase_tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')
paraphrase_model = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase').to(device)

print("PARAPHRASING MODELS LOADED")

summary_tokenizer = AutoTokenizer.from_pretrained("philschmid/bart-large-cnn-samsum")
summary_model = AutoModelForSeq2SeqLM.from_pretrained("philschmid/bart-large-cnn-samsum").to(device)

print("SUMMARY MODELS LOADED")

'evice = torch.device(\'cuda\') if torch.cuda.is_available() else torch.device(\'cpu\')\nprint("STARTING MODELS LOAD")\n\ntranscript_processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")\ntranscript_model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft").to(device)\n\nprint("TRANSCRIPT MODELS LOADED")\n\nparaphrase_tokenizer = BartTokenizer.from_pretrained(\'eugenesiow/bart-paraphrase\')\nparaphrase_model = BartForConditionalGeneration.from_pretrained(\'eugenesiow/bart-paraphrase\').to(device)\n\nprint("PARAPHRASING MODELS LOADED")\n\nsummary_tokenizer = AutoTokenizer.from_pretrained("philschmid/bart-large-cnn-samsum")\nsummary_model = AutoModelForSeq2SeqLM.from_pretrained("philschmid/bart-large-cnn-samsum").to(device)\n\nprint("SUMMARY MODELS LOADED")'

In [22]:

def fetch_audio(url = None, filepath = None):
    print('-----------------------------------EXTRACTING AUDIO-----------------------------------------------')
    if url is not None:
        yt = YouTube(url).streams.filter(progressive=True, file_extension='mp4')
        filepath = yt.first().download()
    
        clip = mp.VideoFileClip(filepath)
        clip.audio.write_audiofile('converted.wav')
    
    elif filepath is not None:
        clip = mp.VideoFileClip(filepath)
        clip.audio.write_audiofile('converted.wav')

    os.remove(filepath)
    return filepath.split('.')[0]

def fetch_chunks(url = None, filepath = None):
    if url is not None:
        name = fetch_audio(url = url)
    
    elif filepath is not None:
        name = fetch_audio(filepath = filepath)
    
    print('-----------------------------------SPLITTING AUDIO------------------------------------------------')
    speech, sr = librosa.load('converted.wav', sr=16000)

    buffer = 30 * sr
    samples_total = len(speech)
    samples_wrote = 0
    counter = 1

    if not os.path.exists('chunks'):
        os.makedirs('chunks')
        
    pbar = tqdm(total=int(samples_total//buffer))
    while samples_wrote < samples_total:
        if buffer > (samples_total - samples_wrote):
            buffer = samples_total - samples_wrote

        block = speech[samples_wrote : (samples_wrote + buffer)]
        out_filename = f'chunks/split_{counter}.wav'

        sf.write(out_filename, block, sr)
        counter += 1

        samples_wrote += buffer
        pbar.update(1)
    os.remove('converted.wav')
    return name

def fetch_transcripts(url = None, filepath = None):
    if url is not None:
        name = fetch_chunks(url = url)
    
    elif filepath is not None:
        name = fetch_chunks(filepath = filepath)
    
    print('-----------------------------------EXTRACTING TRANSCRIPTS----------------------------------------')
    
    text = ""
    for path in tqdm(os.listdir('chunks')):
        data, sampling_rate = librosa.load('chunks/' + path, sr=16000)
        inputs = transcript_processor(data, 
                                      sampling_rate = sampling_rate, 
                                      return_tensors="pt")
        
        inputs = {k:v.to(device) for k, v in inputs.items()}
    
        with torch.no_grad():
            logits = transcript_model(**inputs).logits

        ids = torch.argmax(logits, dim=-1)
        transcription = transcript_processor.batch_decode(ids)[0]
        text += transcription + " "
    
    shutil.rmtree('chunks')
    return name, text

def paraphrase_transcript(url):
    if url is not None:
        name, transcript = fetch_transcripts(url = url)
    
    elif filepath is not None:
        name, transcript = fetch_transcripts(filepath = filepath)
    print(transcript)
    print()
    print()
    print()
    batch = paraphrase_tokenizer(transcript, return_tensors='pt')
    generated_ids = paraphrase_model.generate(batch['input_ids'].to(device))
    
    return name, paraphrase_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

def summarize(url = None, filepath = None):
    if url is not None:
        name, text = fetch_transcripts(url = url)
    
    elif filepath is not None:
        name, text = fetch_transcripts(filepath = filepath)
    
    length = len(text.split())
    
    print('-----------------------------------EXTRACTING SUMMARY----------------------------------------')

    tokens = summary_tokenizer(text.lower(), truncation=True, padding="longest", return_tensors="pt")
    
    tokens = {
        'input_ids': tokens['input_ids'].to(device), 
        'attention_mask': tokens['attention_mask'].to(device)
    }

    summary = summary_model.generate(**tokens)
    return name.split('/')[-1], summary_tokenizer.decode(summary[0], 
                                  skip_special_tokens = True,
                                  temperature = 0.7,
                                  no_repeat_ngram_size=2)

In [24]:
summarize('https://www.youtube.com/watch?v=fj5BcN6Blks')

-----------------------------------EXTRACTING AUDIO-----------------------------------------------
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explici

MoviePy - Done.
-----------------------------------SPLITTING AUDIO------------------------------------------------


-----------------------------------EXTRACTING TRANSCRIPTS----------------------------------------




THIS IS A LIBRIVOX RECORDING ALL LIBRIVOX RECORDINGS ARE IN THE PUBLIC DOMAIN FOR MORE INFORMATION OR TO VOLUNTEER PLEASE VISIT LIBRIVOX DOT ORG THREE QUESTIONS BY LEO TOLSTOY FROM THE COLLECTION WHAT MEN LIVE BY AND OTHER TALES TRANSLATED BY L AND A MAURD IT ONCE OCCURRED TO A CERTAIN KING THAT IF HE ALWAYS KNEW THE RIGHT TIME TO BEGIN EVERYTHING IF HE KNEW WHO WERE THE RIGHT PEOPLE TO LISTEN TO AND WHOM TO AVOID AND ABOVE ALL IF HE ALWAYS KNEW WHAT WAS THE MOST IMPORTANT THING TO DO HE WOULD NEVER FAIL IN ANYTHING HE MIGHT UNDERTAKE AND THIS THOUGHT HAVING OCCURRED TO HIM HE HAD IT PROCLAIMED THROUGHOUT HIS KINGDOM THAT HE WOULD GIVE A GREAT REWARD TO ANYONE WHO WOULD TEACH HIM WHAT WAS THE RIGHT TIME FOR EVERY ACTION AND WHO WERE THE MOST NECESSARY PEOPLE AND HOW HE MIGHT KNOW WHAT WAS THE MOST IMPORTANT THING TO DO AND LEARNED MEN CAME TO THE KING BUT THEY ALL ANSWERED HIS QUESTIONS DIFFERENTLY IN REPLY TO THE FIRST QUESTION SOME SAID THAT TO KNOW THE RIGHT TIME FOR EVERY ACTION 

('TOLSTOY - The Three Questions by Leo Tolstoy - Short story audiobook - FAB',
 "A king wanted to know what was the right time to begin everything and who were the right people to listen to and whom to avoid. He wanted to consult a hermit. The hermit was digging the ground in front of his hut. He didn't give him the spade. The king went to the hermit and asked him three questions. The answers were different.")

In [38]:
import os
import torch
from glob import glob

from pytube import YouTube
import moviepy.editor as mp

import torch
import librosa
import shutil
import soundfile as sf
from tqdm import tqdm


from transformers import Wav2Vec2Processor, HubertForCTC,\
                        BartForConditionalGeneration, BartTokenizer,\
                        AutoModelForSeq2SeqLM, AutoTokenizer

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("STARTING MODELS LOAD")

transcript_processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")
transcript_model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft").to(device)

print("TRANSCRIPT MODELS LOADED")

paraphrase_tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')
paraphrase_model = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase').to(device)

print("PARAPHRASING MODELS LOADED")

summary_tokenizer = AutoTokenizer.from_pretrained("philschmid/bart-large-cnn-samsum")
summary_model = AutoModelForSeq2SeqLM.from_pretrained("philschmid/bart-large-cnn-samsum").to(device)

print("SUMMARY MODELS LOADED")


def fetch_audio(url = None, filepath = None):
    print('-----------------------------------EXTRACTING AUDIO-----------------------------------------------')
    if url is not None:
        yt = YouTube(url).streams.filter(progressive=True, file_extension='mp4')
        filepath = yt.first().download()
    
        clip = mp.VideoFileClip(filepath)
        clip.audio.write_audiofile('converted.wav')
    
    elif filepath is not None:
        clip = mp.VideoFileClip(filepath)
        clip.audio.write_audiofile('converted.wav')

    os.remove(filepath)
    return filepath.split('.')[0]

def fetch_chunks(url = None, filepath = None):
    #if url is not None:
    #    name = fetch_audio(url = url)
    
    #elif filepath is not None:
    #    name = fetch_audio(filepath = filepath)
    name = ''
    print('-----------------------------------SPLITTING AUDIO------------------------------------------------')
    speech, sr = librosa.load('converted.wav', sr=16000)

    buffer = 30 * sr
    samples_total = len(speech)
    samples_wrote = 0
    counter = 1

    if not os.path.exists('chunks'):
        os.makedirs('chunks')
        
    pbar = tqdm(total=int(samples_total//buffer))
    while samples_wrote < samples_total:
        if buffer > (samples_total - samples_wrote):
            buffer = samples_total - samples_wrote

        block = speech[samples_wrote : (samples_wrote + buffer)]
        out_filename = f'chunks/split_{counter}.wav'

        sf.write(out_filename, block, sr)
        counter += 1

        samples_wrote += buffer
        pbar.update(1)
    #os.remove('converted.wav')
    return name

def fetch_transcripts(url = None, filepath = None):
    if url is not None:
        name = fetch_chunks(url = url)
    
    elif filepath is not None:
        name = fetch_chunks(filepath = filepath)
    
    print('-----------------------------------EXTRACTING TRANSCRIPTS----------------------------------------')
    
    text = ""
    for path in tqdm(os.listdir('chunks')):
        data, sampling_rate = librosa.load('chunks/' + path, sr=16000)
        inputs = transcript_processor(data, 
                                      sampling_rate = sampling_rate, 
                                      return_tensors="pt")
        
        inputs = {k:v.to(device) for k, v in inputs.items()}
    
        with torch.no_grad():
            logits = transcript_model(**inputs).logits

        ids = torch.argmax(logits, dim=-1)
        transcription = transcript_processor.batch_decode(ids)[0]
        text += transcription + " "
    
    shutil.rmtree('chunks')
    return name, text

def paraphrase_transcript(url):
    if url is not None:
        name, transcript = fetch_transcripts(url = url)
    
    elif filepath is not None:
        name, transcript = fetch_transcripts(filepath = filepath)
    print(transcript)
    print()
    print()
    print()
    batch = paraphrase_tokenizer(transcript, return_tensors='pt')
    generated_ids = paraphrase_model.generate(batch['input_ids'].to(device))
    
    return name, paraphrase_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

def summarize(url = None, filepath = None):
    if url is not None:
        name, text = fetch_transcripts(url = url)
    
    elif filepath is not None:
        name, text = fetch_transcripts(filepath = filepath)
    
    length = len(text.split())
    
    print('-----------------------------------EXTRACTING SUMMARY----------------------------------------')

    tokens = summary_tokenizer(text.lower(), truncation=True, padding="longest", return_tensors="pt")
    
    tokens = {
        'input_ids': tokens['input_ids'].to(device), 
        'attention_mask': tokens['attention_mask'].to(device)
    }

    summary = summary_model.generate(**tokens)
    return name.split('/')[-1], summary_tokenizer.decode(summary[0], 
                                  skip_special_tokens = True,
                                  temperature = 0.7,
                                  no_repeat_ngram_size=2)

STARTING MODELS LOAD
TRANSCRIPT MODELS LOADED
PARAPHRASING MODELS LOADED
SUMMARY MODELS LOADED


In [40]:
summarize(url = '')

-----------------------------------SPLITTING AUDIO------------------------------------------------


11it [00:00, 100.33it/s]              


-----------------------------------EXTRACTING TRANSCRIPTS----------------------------------------


100%|██████████| 11/11 [00:05<00:00,  1.95it/s]


-----------------------------------EXTRACTING SUMMARY----------------------------------------


('',
 'Companies rated high on the quality of their customer service keep customers longer and have lower sales and marketing costs. Research shows that employees leave organizations because of poor relationships with managers. The key to service excellence is to find and retain quality people and to show appreciation for their hard work.')

In [1]:
from utils import summarize

STARTING MODELS LOAD
TRANSCRIPT MODELS LOADED
PARAPHRASING MODELS LOADED
SUMMARY MODELS LOADED


In [2]:
summarize('https://www.youtube.com/watch?v=fj5BcN6Blks&ab_channel=FabAudioBooks')

-----------------------------------EXTRACTING AUDIO-----------------------------------------------
MoviePy - Writing audio in converted.wav


MoviePy - Done.
-----------------------------------SPLITTING AUDIO------------------------------------------------


  0%|          | 0/25 [00:00<?, ?it/s]

-----------------------------------EXTRACTING TRANSCRIPTS----------------------------------------


25it [00:02, 10.90it/s]                        

100%|██████████| 25/25 [00:08<00:00,  3.07it/s]


-----------------------------------EXTRACTING SUMMARY----------------------------------------


('TOLSTOY - The Three Questions by Leo Tolstoy - Short story audiobook - FAB',
 "A king wanted to know what was the right time to begin everything and who were the right people to listen to and whom to avoid. He wanted to consult a hermit. The hermit was digging the ground in front of his hut. He didn't give him the spade. The king went to the hermit and asked him three questions to find the answers.")